<a href="https://colab.research.google.com/github/ramanath-naik/machine_learning/blob/main/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dense

class TransformerEncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(TransformerEncoderLayer, self).__init__()
        self.mha = MultiHeadAttention(num_heads, d_model)
        self.ffn = tf.keras.Sequential([
            Dense(dff, activation='relu'),
            Dense(d_model)
        ])

        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training):
        attn_output = self.mha(x, x, x)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2


In [3]:
class TransformerDecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(TransformerDecoderLayer, self).__init__()
        self.mha1 = MultiHeadAttention(num_heads, d_model)
        self.mha2 = MultiHeadAttention(num_heads, d_model)

        self.ffn = tf.keras.Sequential([
            Dense(dff, activation='relu'),
            Dense(d_model)
        ])

        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.layernorm3 = LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)

    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3, attn_weights_block1, attn_weights_block2

In [5]:
import tensorflow as tf

class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
                 target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()
        self.encoder = Encoder(num_layers, d_model, num_heads, dff,
                               input_vocab_size, pe_input, rate)
        self.decoder = Decoder(num_layers, d_model, num_heads, dff,
                               target_vocab_size, pe_target, rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)

    def call(self, inp, tar, training, enc_padding_mask,
             look_ahead_mask, dec_padding_mask):
        enc_output = self.encoder(inp, training, enc_padding_mask)
        dec_output, attention_weights = self.decoder(
            tar, enc_output, training, look_ahead_mask, dec_padding_mask)

        final_output = self.final_layer(dec_output)
        return final_output, attention_weights


In [6]:
pip install transformers

In [7]:
from transformers import BertTokenizer, TFBertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relatio

In [8]:
# Example sentences
sentences = ["I love this product!", "This is a bad product."]

# Tokenize sentences
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="tf")

In [14]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

# Example sentences
sentences = ["I love this product!", "This is a bad product."]

# Tokenize sentences
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="tf")

# Convert BatchEncoding to TensorFlow tensors
input_ids = tf.constant(inputs['input_ids'])
attention_mask = tf.constant(inputs['attention_mask'])

# Define labels
labels = tf.convert_to_tensor([1, 0], dtype=tf.float32)  # Assuming binary labels

# Define input layers explicitly
input_ids_input = tf.keras.Input(shape=(None,), dtype='int32', name='input_ids')
attention_mask_input = tf.keras.Input(shape=(None,), dtype='int32', name='attention_mask')

# Load the pre-trained BERT model
bert = TFBertModel.from_pretrained('bert-base-uncased')

# Process inputs through BERT model
bert_output = bert([input_ids_input, attention_mask_input])

# Add a classification layer on top
x = bert_output.last_hidden_state[:, 0, :]
x = tf.keras.layers.Dense(128, activation='relu')(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# Construct the final model
fine_tuned_model = tf.keras.Model(inputs=[input_ids_input, attention_mask_input], outputs=output)

# Compile the model
fine_tuned_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
fine_tuned_model.fit([input_ids, attention_mask], labels, epochs=3, batch_size=32)



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/3


1/1 [==============================] - 38s 38s/step - loss: 0.7469 - accuracy: 0.5000
Epoch 2/3
1/1 [==============================] - 2s 2s/step - loss: 0.7747 - accuracy: 0.5000
Epoch 3/3
1/1 [==============================] - 1s 1s/step - loss: 0.9367 - accuracy: 0.5000


In [25]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

# Example sentences
test_sentences = ["This is bad product", "Absolutely fantastic!"]

# Tokenize test sentences
test_inputs = tokenizer(test_sentences, padding=True, truncation=True, return_tensors="tf")

# Convert BatchEncoding to TensorFlow tensors
test_input_ids = tf.constant(test_inputs['input_ids'])
test_attention_mask = tf.constant(test_inputs['attention_mask'])

# Predict with the fine-tuned model
predictions = fine_tuned_model.predict([test_input_ids, test_attention_mask])

# Interpret the predictions
for sentence, prediction in zip(test_sentences, predictions):
    sentiment = "Positive" if prediction > 0.55 else "Negative"
    print(f"Sentence: '{sentence}' - Prediction: {prediction} - Sentiment: {sentiment}")



1/1 [==============================] - 0s 172ms/step
Sentence: 'This is bad product' - Prediction: [0.5497155] - Sentiment: Negative
Sentence: 'Absolutely fantastic!' - Prediction: [0.5508225] - Sentiment: Positive
